In [1]:
import os
import re
import pathlib
import string
from tqdm import tqdm
import nltk
import numpy as np
import pandas as pd
tqdm.pandas()
import tensorflow as tf
from tensorflow import keras

C:\Users\vinne\Anaconda3\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
tf.__version__

'2.0.0'

In [2]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

True

Load Pretrained [Glove word embeddings](http://nlp.stanford.edu/projects/glove/) 

**`wget` & `unzip` works on Linux interface machines (like Google Colab) or else you can download the glove embeddings file from [here](http://www-nlp.stanford.edu/data/glove.840B.300d.zip)**

In [3]:
!wget http://www-nlp.stanford.edu/data/glove.840B.300d.zip

--2020-07-30 09:24:13--  http://www-nlp.stanford.edu/data/glove.840B.300d.zip
Resolving www-nlp.stanford.edu (www-nlp.stanford.edu)... 171.64.67.140
Connecting to www-nlp.stanford.edu (www-nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2020-07-30 09:24:13--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip [following]
--2020-07-30 09:24:13--  http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP r

In [4]:
!unzip -q glove.840B.300d.zip

replace glove.840B.300d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [5]:
glove_embeddings_index = {}
with open('/content/glove.840B.300d.txt') as f:
    for line in tqdm(f):
        # word, coefs = line.split(maxsplit=1)
        # coefs = np.fromstring(coefs, "f", sep=" ")
        values = line.split( )
        word = values[0]
        try:
          coefs = np.asarray(values[1:], dtype = 'float32')
        except ValueError:
          pass
        glove_embeddings_index[word] = coefs
f.close()
print("\n")
print(f"Found {len(glove_embeddings_index)} word vectors")

2196017it [02:50, 12894.50it/s]



Found 2195884 word vectors


In [45]:
train_data = pd.read_csv(r'/content/drive/My Drive/Data_disaster_real_tweets/train.csv')
test_data = pd.read_csv(r'/content/drive/My Drive/Data_disaster_real_tweets/test.csv')
print('Training data shape: ', train_data.shape)
train_data.head()

Training data shape:  (7613, 5)


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [46]:
list(train_data.text.values)[:10]

['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
 'Forest fire near La Ronge Sask. Canada',
 "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
 '13,000 people receive #wildfires evacuation orders in California ',
 'Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school ',
 '#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires',
 '#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas',
 "I'm on top of the hill and I can see a fire in the woods...",
 "There's an emergency evacuation happening now in the building across the street",
 "I'm afraid that the tornado is coming to our area..."]

In [47]:
def text_preprocessing(text):
    '''
    Make text lowercase, remove text in square brackets,remove links,remove special characters
    and remove words containing numbers.
    '''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) # remove special chars
    text = re.sub('https?:\S+|www.\S+', '', text)  # remove urls 
    text = re.sub('<.*?>+', '', text)  # remove html tags
    # text = re.sub('[^a-zA-Z]', ' ', text) #remove non alphabetical characters
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
  
    return text

clean the both train & test text per `custom_preprocessor` function

In [48]:
train_data['text'] = train_data['text'].progress_apply(lambda x : text_preprocessing(x))
test_data['text'] = test_data['text'].progress_apply(lambda x : text_preprocessing(x))

100%|██████████| 3263/3263 [00:00<00:00, 32978.19it/s]


In [49]:
list(train_data['text'].values)[:10]

['our deeds are the reason of this  earthquake may allah forgive us all',
 'forest fire near la ronge sask  canada',
 'all residents asked to  shelter in place  are being notified by officers  no other evacuation or shelter in place orders are expected',
 '  people receive  wildfires evacuation orders in california ',
 'just got sent this photo from ruby  alaska as smoke from  wildfires pours into a school ',
 ' rockyfire update    california hwy   closed in both directions due to lake county fire    cafire  wildfires',
 ' flood  disaster heavy rain causes flash flooding of streets in manitou  colorado springs areas',
 'i m on top of the hill and i can see a fire in the woods   ',
 'there s an emergency evacuation happening now in the building across the street',
 'i m afraid that the tornado is coming to our area   ']

In [50]:
from sklearn.model_selection import train_test_split

xtrain,xvalid,ytrain,yvalid = train_test_split(train_data.text.values,train_data.target.values,
                                               test_size = 0.2,
                                               stratify = train_data.target.values,
                                               random_state = 42,
                                               shuffle = True)

In [51]:
# print(type(xtrain))
# print(type(ytrain))

In [52]:
from tensorflow.keras.preprocessing import text,sequence

In [53]:
xtrain_list = list(xtrain)
xvalid_list = list(xvalid)
# xtrain = list(train_data.text.values)
xtest = list(test_data.text.values)

In [54]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None , oov_token="<OOV>")
max_len = 80

token.fit_on_texts(xtrain_list)
xtrain_seq = token.texts_to_sequences(xtrain_list)
xvalid_seq = token.texts_to_sequences(xvalid_list)

# token.fit_on_texts(xtrain)
# xtrain_seq = token.texts_to_sequences(xtrain)
xtest_seq = token.texts_to_sequences(xtest)

# zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq , padding = 'post' , maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq , padding = 'post' , maxlen=max_len)
xtest_pad = sequence.pad_sequences(xtest_seq, padding = 'post', maxlen=max_len)

word_index = token.word_index

In [55]:
len(word_index)

14488

In [56]:
# ytrain = train_data.target.values
ytrain

array([1, 0, 0, ..., 0, 0, 0])

In [57]:
num_tokens = len(word_index) + 1
embedding_dim = 300
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in tqdm(word_index.items()):
    embedding_vector = glove_embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

100%|██████████| 14488/14488 [00:00<00:00, 383720.16it/s]

Converted 11047 words (3441 misses)


In [58]:
num_tokens

14489

## Model with Glove embeddings & LSTM

In [59]:
#LSTM with glove embeddings
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(num_tokens , embedding_dim,
                              embeddings_initializer=keras.initializers.Constant(embedding_matrix),
                              input_length = max_len ,
                              trainable = False),
   tf.keras.layers.SpatialDropout1D(0.3),
   tf.keras.layers.LSTM(200, dropout=0.3, recurrent_dropout = 0.3),
   tf.keras.layers.Dense(1024, activation = 'relu'),
   tf.keras.layers.Dropout(0.8),
   tf.keras.layers.Dense(512, activation = 'relu'),
   tf.keras.layers.Dropout(0.8),
   tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy',tf.keras.metrics.AUC()])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 80, 300)           4346700   
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, 80, 300)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 200)               400800    
_________________________________________________________________
dense_9 (Dense)              (None, 1024)              205824    
_________________________________________________________________
dropout_6 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)              

In [60]:
from tensorflow.keras.callbacks import EarlyStopping
# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')

In [61]:
model.fit(xtrain_pad, y=ytrain, batch_size=512, epochs=100, 
          verbose=1, validation_data = (xvalid_pad,yvalid), callbacks = [earlystop])
# model.fit(xtrain_pad, y=ytrain, batch_size=512, epochs=100, 
#           verbose=1, validation_data = (xvalid_pad,yvalid))
# model.fit(xtrain_pad, y=ytrain, batch_size=512, epochs=100, 
#           verbose=1)

Epoch 1/100
12/12 [==============================] - 5s 439ms/step - loss: 0.6883 - accuracy: 0.5598 - auc_4: 0.4868 - val_loss: 0.6832 - val_accuracy: 0.5706 - val_auc_4: 0.5000
Epoch 2/100
12/12 [==============================] - 5s 418ms/step - loss: 0.6831 - accuracy: 0.5703 - auc_4: 0.5142 - val_loss: 0.6832 - val_accuracy: 0.5706 - val_auc_4: 0.5000
Epoch 3/100
12/12 [==============================] - 5s 413ms/step - loss: 0.6842 - accuracy: 0.5703 - auc_4: 0.4929 - val_loss: 0.6833 - val_accuracy: 0.5706 - val_auc_4: 0.5000
Epoch 4/100
12/12 [==============================] - 5s 419ms/step - loss: 0.6836 - accuracy: 0.5703 - auc_4: 0.5022 - val_loss: 0.6832 - val_accuracy: 0.5706 - val_auc_4: 0.5000
Epoch 5/100
12/12 [==============================] - 5s 402ms/step - loss: 0.6838 - accuracy: 0.5703 - auc_4: 0.5010 - val_loss: 0.6832 - val_accuracy: 0.5706 - val_auc_4: 0.5000
Epoch 6/100
12/12 [==============================] - 5s 402ms/step - loss: 0.6839 - accuracy: 0.5703 - au

In [67]:
model.save("model_with_glove_embeddings_LSTM.h5")

In [63]:
ypred = model.predict_classes(xtest_pad)
ypred

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [0]], dtype=int32)

In [64]:
np.unique(ypred)

array([0, 1], dtype=int32)

In [65]:
# Submission
sample_submission = pd.read_csv(r"/content/drive/My Drive/Data_disaster_real_tweets/sample_submission.csv")
sample_submission["target"] = ypred
sample_submission.head()


,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [66]:
sample_submission.to_csv("submission_glove_word_vecs_LSTM_ES.csv", index=False)

0.80140 -> Kaggle score after submission

## Build a model with Glove embeddings & Bidirectional LSTM

In [68]:
#LSTM with glove embeddings
model_Bi_lstm = tf.keras.Sequential([
    tf.keras.layers.Embedding(num_tokens , embedding_dim,
                              embeddings_initializer=keras.initializers.Constant(embedding_matrix),
                              input_length = max_len ,
                              trainable = False),
   tf.keras.layers.SpatialDropout1D(0.3),
   tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(300, dropout=0.3, recurrent_dropout = 0.3)),
   tf.keras.layers.Dense(1024, activation = 'relu'),
   tf.keras.layers.Dropout(0.8),
   tf.keras.layers.Dense(512, activation = 'relu'),
   tf.keras.layers.Dropout(0.8),
   tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model_Bi_lstm.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy',tf.keras.metrics.AUC()])
model_Bi_lstm.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 80, 300)           4346700   
_________________________________________________________________
spatial_dropout1d_4 (Spatial (None, 80, 300)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 600)               1442400   
_________________________________________________________________
dense_12 (Dense)             (None, 1024)              615424    
_________________________________________________________________
dropout_8 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)              

In [69]:
from tensorflow.keras.callbacks import EarlyStopping
# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=15, verbose=0, mode='auto')

model_Bi_lstm.fit(xtrain_pad, y=ytrain, batch_size=512, epochs=100, 
          verbose=1, validation_data = (xvalid_pad,yvalid), callbacks = [earlystop])
model_Bi_lstm.save('model_with_glove_embeddings_Bidire_LSTM.h5')



Epoch 1/100
12/12 [==============================] - 11s 896ms/step - loss: 0.6305 - accuracy: 0.6340 - auc_5: 0.6810 - val_loss: 0.5022 - val_accuracy: 0.7485 - val_auc_5: 0.8371
Epoch 2/100
12/12 [==============================] - 10s 857ms/step - loss: 0.5113 - accuracy: 0.7672 - auc_5: 0.8204 - val_loss: 0.4509 - val_accuracy: 0.8155 - val_auc_5: 0.8696
Epoch 3/100
12/12 [==============================] - 10s 826ms/step - loss: 0.4669 - accuracy: 0.7982 - auc_5: 0.8459 - val_loss: 0.4284 - val_accuracy: 0.8162 - val_auc_5: 0.8753
Epoch 4/100
12/12 [==============================] - 10s 834ms/step - loss: 0.4535 - accuracy: 0.8028 - auc_5: 0.8566 - val_loss: 0.4204 - val_accuracy: 0.8194 - val_auc_5: 0.8815
Epoch 5/100
12/12 [==============================] - 10s 844ms/step - loss: 0.4564 - accuracy: 0.7961 - auc_5: 0.8551 - val_loss: 0.4383 - val_accuracy: 0.8175 - val_auc_5: 0.8832
Epoch 6/100
12/12 [==============================] - 10s 822ms/step - loss: 0.4413 - accuracy: 0.802

In [70]:
sample_submission = pd.read_csv(r"/content/drive/My Drive/Data_disaster_real_tweets/sample_submission.csv")
sample_submission["target"] = model_Bi_lstm.predict_classes(xtest_pad)
sample_submission.to_csv("submission_glove_word_vecs_Bidire_LSTM_ES.csv", index=False)

**Model with unidirectional LSTM gives better results than with Bidirectional LSTM & now Let's train the entire data set on Unidirectional LSTM model***

## Let's try to fit the model to entire data set

In [34]:
xtrain = list(train_data.text.values)
xtest = list(test_data.text.values)

# using keras tokenizer here
token = text.Tokenizer(num_words=None , oov_token="<OOV>")
max_len = 80

token.fit_on_texts(xtrain)
xtrain_seq = token.texts_to_sequences(xtrain)
xtest_seq = token.texts_to_sequences(xtest)

# zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq , padding = 'post' , maxlen=max_len)
xtest_pad = sequence.pad_sequences(xtest_seq, padding = 'post', maxlen=max_len)

word_index = token.word_index

ytrain = train_data.target.values

num_tokens = len(word_index) + 1
embedding_dim = 300
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in tqdm(word_index.items()):
    embedding_vector = glove_embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

100%|██████████| 16593/16593 [00:00<00:00, 351218.67it/s]

Converted 12386 words (4207 misses)


In [39]:
num_tokens

16594

In [41]:
#LSTM with glove embeddings
model_1 = tf.keras.Sequential([
    tf.keras.layers.Embedding(num_tokens , embedding_dim,
                              embeddings_initializer=keras.initializers.Constant(embedding_matrix),
                              input_length = max_len ,
                              trainable = False),
   tf.keras.layers.SpatialDropout1D(0.3),
   tf.keras.layers.LSTM(200, dropout=0.3, recurrent_dropout = 0.3),
   tf.keras.layers.Dense(1024, activation = 'relu'),
   tf.keras.layers.Dropout(0.8),
   tf.keras.layers.Dense(512, activation = 'relu'),
   tf.keras.layers.Dropout(0.8),
   tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model_1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy',tf.keras.metrics.AUC()])
model_1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 80, 300)           4978200   
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 80, 300)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 200)               400800    
_________________________________________________________________
dense_6 (Dense)              (None, 1024)              205824    
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)              

In [42]:
model_1.fit(xtrain_pad, y=ytrain, batch_size=512, epochs=100, verbose=1)

Epoch 1/100
15/15 [==============================] - 6s 373ms/step - loss: 0.6868 - accuracy: 0.5657 - auc_3: 0.5015
Epoch 2/100
15/15 [==============================] - 6s 382ms/step - loss: 0.6839 - accuracy: 0.5703 - auc_3: 0.4978
Epoch 3/100
15/15 [==============================] - 6s 381ms/step - loss: 0.6825 - accuracy: 0.5703 - auc_3: 0.5093
Epoch 4/100
15/15 [==============================] - 6s 381ms/step - loss: 0.5852 - accuracy: 0.6976 - auc_3: 0.7523
Epoch 5/100
15/15 [==============================] - 6s 381ms/step - loss: 0.5335 - accuracy: 0.7563 - auc_3: 0.8085
Epoch 6/100
15/15 [==============================] - 6s 373ms/step - loss: 0.5029 - accuracy: 0.7889 - auc_3: 0.8255
Epoch 7/100
15/15 [==============================] - 6s 381ms/step - loss: 0.4750 - accuracy: 0.8002 - auc_3: 0.8413
Epoch 8/100
15/15 [==============================] - 6s 373ms/step - loss: 0.4673 - accuracy: 0.8017 - auc_3: 0.8473
Epoch 9/100
15/15 [==============================] - 6s 375ms/st

In [43]:
model_1.save("model_with_glove_embeddings_full_model_LSTM.h5")

# Submission
sample_submission = pd.read_csv(r"/content/drive/My Drive/Data_disaster_real_tweets/sample_submission.csv")
sample_submission["target"] = model_1.predict_classes(xtest_pad)
sample_submission.head()


,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [44]:
sample_submission.to_csv("submission_full_data_glove_word_vecs_LSTM.csv", index=False)

0.79252 -> Kaggle Submission